Welcome back! Today, we'll build a simple web app using `Dash`, developed by the folks at Plotly.

We'll create a message bank where users can submit messages and view randomly selected ones.

## Preparing our Environment

First, let's import the necessary modules: ...

In [5]:
from dash import Dash, html, dcc, callback, Input, Output, State
import dash_bootstrap_components as dbc
import sqlite3

## Building the Dash App

### Define basic UI elements

Our simple app will include:

* A text box for submitting a message.
* A text box for entering the name or handle of the user.
* A “Submit” button to insert messages into the database.
* A viewing area that displays 5 random messages from the database.
* An "Update" button to retrieve random messages.

Before we enable any functions to the buttons, let's first implement this basic interface!

To keep the design simple yet appealing, we’ll use Bootstrap components for Dash. We can also pick a custom font to add a personal touch!

In [10]:
app = Dash(__name__, 
          external_stylesheets=[
              dbc.themes.MINTY,
              "https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&family=Space+Mono:ital,wght@0,400;0,700;1,400&display=swap"
          ])

custom_styles = {
    'title': {
        'font-family': 'Space Mono, monospace',
        'color': '#2C3E50',
        'letter-spacing': '1px'
    },
    'body': {
        'font-family': 'Poppins, sans-serif',
        'background-color': '#F8F9FA'
    },
    'message': {
        'font-family': 'Space Mono, monospace',
        'font-style': 'italic',
        'color': '#34495E'
    }
}

app.title = "Dash Message Bank"

app.layout = dbc.Container([
    
    html.H1('A SIMPLE MESSAGE BANK', 
            className='mt-4 mb-4 text-center',
            style=custom_styles['title']),
    
    # First section: Submit
    html.H2('Submit', 
            className='mb-4',
            style=custom_styles['title']),
    
    # Message input as textarea
    dbc.Label('Your Message:', className='h4'),
    dbc.Textarea(
        id='message',
        className='mb-3 shadow-sm',
        style={
            'font-family': 'Poppins, sans-serif',
            'height': '100px'
        }
    ),
    
    # Name/Handle input
    dbc.Label('Your Name or Handle:', className='h4'),
    dbc.Input(
        id='handle',
        type='text',
        className='mb-3 shadow-sm',
        style={
            'font-family': 'Poppins, sans-serif',
            'max-width': '300px'  # Limit width of username field
        }
    ),
    
    # Submit button
    dbc.Button(
        'SUBMIT', 
        id='submit', 
        color='primary', # Green as in the theme
        className='mt-2 px-4',
        style={'font-family': 'Poppins, sans-serif'}
    ),

    # Logs that show if a message is added successfully
    html.Div(id='feedback', className='mt-3 text-success'),
    
    html.Hr(className='my-4'),

    # Second section: View
    html.H2('View', 
            className='mb-4',
            style=custom_styles['title']),
    
    html.Div(id='messages-display',
             className='p-3',
             style={'background-color': 'white', 'border-radius': '8px'}),
    
    # Update button
    dbc.Button(
        'UPDATE', 
        id='update', 
        className='mt-3 px-4',
        style={
            'font-family': 'Poppins, sans-serif',
            'background-color': '#9B59B6',  # Make it purple!
            'border-color': '#9B59B6',
            'color': 'white'
        }
    )  
], className='p-4', style=custom_styles['body'])

### Set up the database

To begin, we'll create and manage our SQLite database.

We can first define a function `get_message_db()`, which initializes the database if it doesn’t already exist. It will create a "messages" table with two text columns: "handle" and "message."

In [13]:
message_db = None
def get_message_db():
    '''
    Creates or returns a connection to an SQLite database for storing messages.
    Returns: sqlite3 connection to the messages database
    '''
    global message_db
    if not message_db:  # if None or closed
        message_db = sqlite3.connect("messages_db.sqlite", check_same_thread=False)
        cmd = '''CREATE TABLE IF NOT EXISTS messages (handle TEXT, message TEXT)'''
        cursor = message_db.cursor()
        cursor.execute(cmd)
    return message_db

### Insert messages

Next, we can create the `insert_message()` function, which inserts messages into the database while properly handling string quotes. We'll also save changes after each insertion.

In [16]:
def insert_message(handle, message):
    '''
    Inserts a message into the messages database.
    Args:
        handle(str): user's name or handle
        message(str): message content     
    '''
    # Use previous function to get database connection
    global message_db
    db = get_message_db()

    cursor = db.cursor()
    cmd = '''INSERT INTO messages(handle, message) VALUES (?, ?)'''
    cursor.execute(cmd, (handle, message))

    db.commit() # Commit changes to save the insertion
    db.close()
    message_db = None

### Enable submissions

Then, we can define a decorator to handle user input submission.

A decorator or callback function makes the webpage interactive by taking input components and updating output components (essentially child elements in our HTML).

Now, let's create a `submit()` function. Its logic is straightforward: 

* If the button is clicked and both input fields are filled, the message is inserted into the database.
*  A confirmation message is returned and gets displayed in the `feedback` div.

In [19]:
@callback(
    # Output: update the feedback div to show result
    [Output('feedback', 'children'),
    Output('message', 'value'),
    Output('handle', 'value')],
    
    # Input: listen for submit button clicks
    Input('submit', 'n_clicks'),
    
    # State: get current values from input fields
    [State('message', 'value'),
     State('handle', 'value')],
    prevent_initial_call=True
)
def submit(n_clicks, message, handle):
    """
    Callback function to handle message submission and provide user feedback.
    
    Args:
       n_clicks(int): number of times submit button clicked
       handle(str): username from input field
       message(str): user message from input field
   
   Returns:
       str: feedback log to display to user
   """
    # Validate that both fields have values
    if not message or not handle:
        return "Please fill in both message and name fields.", message, handle # Error message to alert the user
    
    # Insert message into database
    try:
        insert_message(handle, message)
        return "Thanks for submitting a message!", "", ""  # Clear the inputs after successful submission
    except sqlite3.Error as e:
        return f"An error occurred: {str(e)}", message, handle

### Retrieve Random Submissions

Before enabling the "Update" feature, we'll create a helper function to fetch a specified number of random messages from the database.

In [22]:
def random_messages(n):
    '''
    Returns n random messages from the database.
    Args:
        n(int): number of random messages to return
    Returns:
        list: list of (handle, message) tuples
    '''
    global message_db
    db = get_message_db()
    cursor = db.cursor()
    cmd = '''SELECT * FROM messages ORDER BY RANDOM() LIMIT ?'''
    cursor.execute(cmd, (n,))
    messages = cursor.fetchall()
    db.close()
    message_db = None
    return messages

### View Random Submissions

Finally, we can write another callback to handle message display:

* If the button is clicked, `random_messages()` will help fetch messages from `messages_db`.
* Then, messages are displayed as a list in the `message-display` div.

Here, I've set the cap at five messages at a time.

In [25]:
@callback(
    Output('messages-display', 'children'),
    Input('update', 'n_clicks'),
    prevent_initial_call=True
)
def view(n_clicks):
    if n_clicks is None:
        return ""
    
    messages = random_messages(5)
    message_elements = []
    for handle, msg in messages:
        message_elements.extend([
            html.P(msg, 
                  className='mb-1',
                  style={'font-family': 'Poppins, sans-serif', 'color': '#2C3E50'}),
            html.P(f"- {handle}", 
                  className='mb-4 fst-italic',
                  style=custom_styles['message'])
        ])
    
    return message_elements

## Testing the App

Now we're all set! Let's try to run the app in the web browser:

In [28]:
if __name__ == '__main__':
    app.run(port=8050, debug=True)

We can submit messages and see a confirmation of successful submission.

After submitting a few messages, we can test the "Update" button to display five random messages on the same page!

Awesome! Now we can either continue to enhance the page's design or deploy it to the cloud! Have fun!